In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping

In [ ]:
#גרסאת גיט
!git --version


git version 2.50.0.windows.1


In [ ]:
#שימוש במודל מאומן YOLOV5
!git clone https://github.com/ultralytics/yolov5 
!pip install -r yolov5/requirements.txt     
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117  


fatal: destination path 'yolov5' already exists and is not an empty directory.


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [ ]:
#הסרת backports והתקנת backports.tarfile
!pip uninstall backports
!pip install backports.tarfile


In [5]:
import torch

# טוען את מודל YOLOv5s המאומן מראש (הגרסה הקטנה)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


Using cache found in C:\Users\vlsi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-6-26 Python-3.8.8 torch-2.4.1+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# חיבור לסימולטור 
client = carla.Client('localhost', 2000)

In [ ]:
#טעינת עיר בסימולציית קרלה
client.load_world('Town01')

In [ ]:
#הגדרת סביבה בסימולצייה
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [ ]:
#הגדרת רכב לסימולצייה
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [ ]:
#מיקום הרכב במיקום רנדומלי
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


In [ ]:
#העברת תצוגת הסימולטור לרכב
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)

In [ ]:
# הגדרות מיקום מצלמת הנהג בתוך הרכב
CAMERA_POS_X = 0.4   # מעט קדימה מהמושב
CAMERA_POS_Y = 0.0   # מרכז הרכב
CAMERA_POS_Z = 1.2   # גובה העיניים של נהג ממוצע

# הגדרת בלופרינט של המצלמה
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')
camera_bp.set_attribute('fov', '90')  # שדה ראייה ריאליסטי

# מיקום המצלמה בתוך הרכב
camera_init_trans = carla.Transform(
    carla.Location(x=CAMERA_POS_X, y=CAMERA_POS_Y, z=CAMERA_POS_Z),
    carla.Rotation(pitch=0.0, yaw=0.0, roll=0.0)  # מבט ישר קדימה
)

# יצירת המצלמה ומצמידים אותה לרכב
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# הפעלת הנהיגה האוטונומית
vehicle.set_autopilot(True)

# פונקציית callback של המצלמה
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# הגדרת מאגר לתמונות מהמצלמה
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

#  פונקציה שמזיזה את הספקטייטור אחרי המצלמה בזמן אמת
def follow_camera(camera_actor):
    spectator = world.get_spectator()
    def tick(world_snapshot):
        transform = camera_actor.get_transform()
        spectator.set_transform(transform)
    world.on_tick(tick)

# קריאה לפונקציה כדי להתחיל את המעקב
follow_camera(camera)


In [ ]:
import cv2
import numpy as np
import math
#סיווג אור הרמזור בסימולצייה
def classify_traffic_light_state(cropped_img):
    hsv = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2HSV)

    red_lower1 = np.array([0, 100, 100])
    red_upper1 = np.array([10, 255, 255])
    red_lower2 = np.array([160, 100, 100])
    red_upper2 = np.array([179, 255, 255])

    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([30, 255, 255])

    green_lower = np.array([40, 100, 100])
    green_upper = np.array([90, 255, 255])

    mask_red1 = cv2.inRange(hsv, red_lower1, red_upper1)
    mask_red2 = cv2.inRange(hsv, red_lower2, red_upper2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)

    mask_yellow = cv2.inRange(hsv, yellow_lower, yellow_upper)
    mask_green = cv2.inRange(hsv, green_lower, green_upper)

    red_count = cv2.countNonZero(mask_red)
    yellow_count = cv2.countNonZero(mask_yellow)
    green_count = cv2.countNonZero(mask_green)

    max_count = max(red_count, yellow_count, green_count)

    if max_count == red_count:
        return 'red'
    elif max_count == yellow_count:
        return 'yellow'
    elif max_count == green_count:
        return 'green'
    else:
        return ''

#חישוב מרחק
def calculate_distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)

#קבלת מיקום הרכב מנתוני הסימולציה
def get_car_position(vehicle):
    loc = vehicle.get_location()
    return (loc.x, loc.y, loc.z)

#קבלת מיקומי הרמזורים מסימולציית הקרלה
def get_traffic_lights_positions(world):
    actors = world.get_actors()
    traffic_lights = actors.filter('traffic.traffic_light')
    positions = []
    for tl in traffic_lights:
        loc = tl.get_location()
        positions.append((loc.x, loc.y, loc.z))
    return positions

#מציאת המיקום של הרמזור הכי קרוב לרכב
def find_closest_traffic_light(car_position, traffic_lights_positions):
    min_distance = float('inf')
    closest_tl = None
    for tl_pos in traffic_lights_positions:
        dist = calculate_distance(car_position, tl_pos)
        if dist < min_distance:
            min_distance = dist
            closest_tl = tl_pos
    return closest_tl, min_distance

#זיהוי רמזורים והצבע של הרמזור
def detect_traffic_lights_and_color(image_bgra, model, vehicle, world, conf_threshold=0.7):
    # המרה ל-RGB
    image_rgb = cv2.cvtColor(image_bgra, cv2.COLOR_BGRA2RGB)
    h, w, _ = image_rgb.shape

    # ריצת המודל על התמונה
    results = model(image_rgb)
    df = results.pandas().xyxy[0]

    # מיקום הרכב והרמזורים בעולם
    car_position = get_car_position(vehicle)
    traffic_lights_positions = get_traffic_lights_positions(world)
    closest_tl_pos, closest_distance = find_closest_traffic_light(car_position, traffic_lights_positions)

    # קביעת מצב ברירת מחדל
    state = ''
    # מציאת הרמזור הראשון שעובר סף ביטחון
    for _, row in df.iterrows():
        if row['name'] == 'traffic light' and row['confidence'] > conf_threshold:
            x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
            cropped = image_rgb[y1:y2, x1:x2]
            state = classify_traffic_light_state(cropped)
            break

    distance_text = f"{closest_distance:.2f}m" if closest_distance is not None else 'N/A'

    # בחירת טקסט וצבע לפי מצב הרמזור
    if state == 'red':
        text = f"STOP! {distance_text}"
        color = (0, 0, 255)
    elif state == 'green':
        text = f"DRIVE NOW! {distance_text}"
        color = (0, 255, 0)
    else:
        text = f""
        color = (255, 255, 255)

    # כתיבת הטקסט בפינה השמאלית–תחתונה
    text_pos = (10, h - 10)
    cv2.putText(image_rgb, text, text_pos,
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    return image_rgb


In [ ]:
while True:
      
    # תצוגה עם imshow
    cv2.imshow('All cameras',camera_data['image'])
    # עיבוד תמונה לזיהוי רמזורים אדומים
    processed_img = detect_traffic_lights_and_color(camera_data['image'],model,vehicle, world)

    # תצוגה
    cv2.imshow('Traffic Light Detection', processed_img)

    
    # הפסק לולאה עם לחיצת q מהמשתמש
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

C:\Users\vlsi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\vlsi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\vlsi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\vlsi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\vlsi/.cache\torch\hub\ultralytics_yolov5_master\models\comm

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [ ]:
# ניקוי הסימולטור

camera.stop() 
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()